In [1]:
import os 
from dotenv import load_dotenv
import requests
import json
import pandas as pd
from datetime import datetime
import pytz

In [2]:
load_dotenv()

api_key = os.environ.get("API_KEY")

In [5]:
coor = {
    "saimai": {"lat": "13.896", "lon": "100.672"},
    "pathumwan": {"lat": "13.740", "lon": "100.535"}
}

parts = "minutely,hourly,daily,alerts"
units = "metric"

In [8]:
weather_url = f"https://api.openweathermap.org/data/3.0/onecall?lat={coor['saimai']['lat']}&lon={coor['saimai']['lon']}&exclude={parts}&units={units}&appid={api_key}"
air_url = f"https://api.openweathermap.org/data/2.5/air_pollution?lat={coor['saimai']['lat']}&lon={coor['saimai']['lon']}&appid={api_key}"

In [9]:
weather_response = requests.get(weather_url)
air_response = requests.get(air_url)

In [10]:
print(weather_response)
print(air_response)

<Response [200]>
<Response [200]>


In [13]:
weather_result = weather_response.json()
air_result = air_response.json()

In [16]:
print(weather_result)
print(air_result)

{'lat': 13.896, 'lon': 100.672, 'timezone': 'Asia/Bangkok', 'timezone_offset': 25200, 'current': {'dt': 1683818347, 'sunrise': 1683759119, 'sunset': 1683804929, 'temp': 30.9, 'feels_like': 34.16, 'pressure': 1010, 'humidity': 58, 'dew_point': 21.67, 'uvi': 0, 'clouds': 20, 'visibility': 10000, 'wind_speed': 2.57, 'wind_deg': 170, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'rain': {'1h': 0.15}}}
{'coord': {'lon': 100.672, 'lat': 13.896}, 'list': [{'main': {'aqi': 5}, 'components': {'co': 3685, 'no': 109.97, 'no2': 45.93, 'o3': 0, 'so2': 87.74, 'pm2_5': 84.79, 'pm10': 128.86, 'nh3': 31.16}, 'dt': 1683818347}]}


In [74]:
weather_dict = {}

thai_tz = pytz.timezone("Asia/Bangkok")

time_list = ["dt", "sunrise", "sunset"]

for key in weather_result["current"].keys():
    if key in time_list:
        unix_time = weather_result["current"][key]
        local_time = datetime.fromtimestamp(unix_time, thai_tz).strftime("%Y-%m-%d %H:%M:%S")
        weather_dict[key] = local_time
    elif key not in time_list + ["weather", "wind_gust", "rain"]:
        weather_dict[key] = weather_result["current"][key]
    elif key == "weather":
        main = weather_result["current"][key][0]["main"]
        description = weather_result["current"][key][0]["description"]
        weather_dict["weather_main"] = main
        weather_dict["weather_description"] = description
    elif key == "wind_gust":
        wind_gust = weather_result["current"][key]
        weather_dict["wind_gust"] = wind_gust
    elif key == "rain":
        rain = weather_result["current"][key]["1h"]
        weather_dict["rain"] = rain

In [76]:
air_dict = {}

air_dict["aqi"] = air_result["list"][0]["main"]["aqi"]

for component in air_result["list"][0]["components"]:
    air_dict[component] = air_result["list"][0]["components"][component]

In [75]:
weather_dict

{'dt': '2023-05-11 22:19:07',
 'sunrise': '2023-05-11 05:51:59',
 'sunset': '2023-05-11 18:35:29',
 'temp': 30.9,
 'feels_like': 34.16,
 'pressure': 1010,
 'humidity': 58,
 'dew_point': 21.67,
 'uvi': 0,
 'clouds': 20,
 'visibility': 10000,
 'wind_speed': 2.57,
 'wind_deg': 170,
 'weather_main': 'Rain',
 'weather_description': 'light rain',
 'rain': 0.15}

In [77]:
air_dict

{'aqi': 5,
 'co': 3685,
 'no': 109.97,
 'no2': 45.93,
 'o3': 0,
 'so2': 87.74,
 'pm2_5': 84.79,
 'pm10': 128.86,
 'nh3': 31.16}

In [82]:
pd.DataFrame(weather_dict, index=[0])

,dt,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,weather_main,weather_description,rain
0,2023-05-11 22:19:07,2023-05-11 05:51:59,2023-05-11 18:35:29,30.9,34.16,1010,58,21.67,0,20,10000,2.57,170,Rain,light rain,0.15


In [86]:
pd.DataFrame(air_dict, index=[0])

,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,5,3685,109.97,45.93,0,87.74,84.79,128.86,31.16
